In [38]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
df=pd.read_csv("data1.csv")

In [5]:
df

,house_id,num_rooms,location,size_sqft,age,num_bathrooms,parking,garden,floor,price
0,1,7,Countryside,480,17,3,0,0,3,172841
1,2,4,Uptown,9712,23,4,1,0,2,1978472
2,3,8,Countryside,4442,92,1,0,1,3,885606
3,4,5,Industrial Area,3923,30,1,2,0,7,809692
4,5,7,Countryside,5029,57,2,3,1,1,1036880
...,...,...,...,...,...,...,...,...,...,...
9995,9996,1,Industrial Area,7344,2,2,0,0,3,1480403
9996,9997,7,Suburb,4264,91,1,0,1,8,852594
9997,9998,6,Countryside,4911,22,3,3,0,8,1053987
9998,9999,9,Downtown,6059,35,2,2,0,7,1292688


In [11]:
df_encoded = pd.get_dummies(df, columns=['location']).astype(int)

In [23]:
df1=df_encoded.to_csv("data4.csv")

In [26]:
df2=pd.read_csv("data4.csv")

In [27]:
df2

,Unnamed: 0,house_id,num_rooms,size_sqft,age,num_bathrooms,parking,garden,floor,price,location_Countryside,location_Downtown,location_Industrial Area,location_Suburb,location_Uptown
0,0,1,7,480,17,3,0,0,3,172841,1,0,0,0,0
1,1,2,4,9712,23,4,1,0,2,1978472,0,0,0,0,1
2,2,3,8,4442,92,1,0,1,3,885606,1,0,0,0,0
3,3,4,5,3923,30,1,2,0,7,809692,0,0,1,0,0
4,4,5,7,5029,57,2,3,1,1,1036880,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,9996,1,7344,2,2,0,0,3,1480403,0,0,1,0,0
9996,9996,9997,7,4264,91,1,0,1,8,852594,0,0,0,1,0
9997,9997,9998,6,4911,22,3,3,0,8,1053987,1,0,0,0,0
9998,9998,9999,9,6059,35,2,2,0,7,1292688,0,1,0,0,0


In [28]:
numeric_cols = df2.select_dtypes(include=['int64', 'float64']).columns.tolist()

In [30]:
minmax = MinMaxScaler()
df_normalized = df2.copy()
df_normalized[numeric_cols] = minmax.fit_transform(df2[numeric_cols])

In [32]:
scaler = StandardScaler()
df_standardized = df2.copy()
df_standardized[numeric_cols] = scaler.fit_transform(df2[numeric_cols])

In [36]:
df_normalized.to_csv("data4_normalized.csv", index=False)
df_standardized.to_csv("data4_standardized.csv", index=False)

In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression

# Load your dataset
df2 = pd.read_csv("data4.csv")

# Separate features (X) and target (y)
X = df2.drop(columns=['price'])   # all columns except price
y = df2['price']                  # target column

# --- Feature Selection ---
selector = SelectKBest(score_func=f_regression, k=5)   # pick top 5 features
X_new = selector.fit_transform(X, y)

# Get selected feature names
selected_features = X.columns[selector.get_support()]
print("\nSelected Features:", selected_features.tolist())

# Keep only selected features
X = X[selected_features]

# --- Train/Test Split ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("\nTraining set shape:", X_train.shape, y_train.shape)
print("Testing set shape:", X_test.shape, y_test.shape)

# --- Model Training ---
model = LinearRegression()
model.fit(X_train, y_train)

# Print coefficients
coefficients = pd.DataFrame({
    "Feature": selected_features,
    "Coefficient": model.coef_
})

print("\nModel Coefficients:")
print(coefficients)

# Print intercept
print("\nIntercept (b0):", model.intercept_)

# Evaluate model
print("\nTraining R^2 Score:", model.score(X_train, y_train))
print("Testing R^2 Score:", model.score(X_test, y_test))


# --- Prediction with Input Validation ---
print("\n--- House Price Prediction ---")

user_input = {}

for feature in selected_features:
    while True:
        try:
            value = float(input(f"Enter value for {feature}: "))

            # Validation rules
            if value < 0:
                print(f"❌ Invalid input: {feature} cannot be negative. Try again.")
                continue

            # (Optional) Example rule: room count must be reasonable
            if "room" in feature.lower() and value > 50:
                print(f"❌ Invalid input: {feature} seems too large. Try again.")
                continue

            user_input[feature] = [value]
            break

        except ValueError:
            print("❌ Please enter a valid number.")

# Convert input to DataFrame
user_df = pd.DataFrame(user_input)

# Predict price
predicted_price = model.predict(user_df)[0]

print(f"\n✅ Predicted House Price: ${predicted_price:,.2f}")



Selected Features: ['Unnamed: 0', 'num_rooms', 'size_sqft', 'age', 'num_bathrooms']

Training set shape: (8000, 5) (8000,)
Testing set shape: (2000, 5) (2000,)

Model Coefficients:
         Feature   Coefficient
0     Unnamed: 0      0.003167
1      num_rooms  10032.293636
2      size_sqft    199.931503
3            age   -996.131543
4  num_bathrooms   4980.227711

Intercept (b0): 8042.991926928749

Training R^2 Score: 0.9998192196354321
Testing R^2 Score: 0.9998107872498372

--- House Price Prediction ---


Enter value for Unnamed: 0:  2
Enter value for num_rooms:  4
Enter value for size_sqft:  2200
Enter value for age:  2
Enter value for num_bathrooms:  2



✅ Predicted House Price: $495,989.67
